In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [3]:
lengt = 7

In [4]:
data_path = 'C:/Santosh/AI/K-Comp/Ion/Data/wdwk/'
train_signal_file = data_path + 'train_signal.npy'
train_channels_file = data_path + 'train_channels.npy'
test_signal_file = data_path + 'test_signal.npy'

train_signal = np.load(train_signal_file)
train_channels = np.load(train_channels_file)
test_signal = np.load(test_signal_file)

test_model_signal = np.zeros((5,1000000))
test_model_channel = np.zeros((5,1000000))
test_model_signal[0][:500000] = train_signal[0].flatten()
test_model_signal[0][500000:] = train_signal[1].flatten()
test_model_signal[1][:500000] = train_signal[2].flatten()
test_model_signal[1][500000:] = train_signal[6].flatten()
test_model_signal[2][:500000] = train_signal[3].flatten()
test_model_signal[2][500000:] = train_signal[7].flatten()
test_model_signal[3][:500000] = train_signal[4].flatten()
test_model_signal[3][500000:] = train_signal[9].flatten()
test_model_signal[4][:500000] = train_signal[5].flatten()
test_model_signal[4][500000:] = train_signal[8].flatten()


test_model_channel[0][:500000] = train_channels[0].flatten()
test_model_channel[0][500000:] = train_channels[1].flatten()
test_model_channel[1][:500000] = train_channels[2].flatten()
test_model_channel[1][500000:] = train_channels[6].flatten()
test_model_channel[2][:500000] = train_channels[3].flatten()
test_model_channel[2][500000:] = train_channels[7].flatten()
test_model_channel[3][:500000] = train_channels[4].flatten()
test_model_channel[3][500000:] = train_channels[9].flatten()
test_model_channel[4][:500000] = train_channels[5].flatten()
test_model_channel[4][500000:] = train_channels[8].flatten()



In [ ]:
from sklearn.svm import SVC
models = []

specs = [[1.2,1],[0.1,1],[0.5,1],[7,0.01],[10,0.1]]


for k in range (5):
  if(k==2 or k==4):
    lengt = 3
  elif(k==3):
    lengt = 13
  else:
    lengt = 7
  print("starting ", k)
  x = test_model_signal[k].flatten()
  y = test_model_channel[k].flatten()
    
  y = np.array(y).astype(int)
  
  new_x = np.zeros((x.shape[0],lengt))
  for i in range(x.shape[0]):
        for j in range(-lengt//2,lengt//2 + 1):
            new_x[i][j + lengt//2] = x[(i +j + 1000000)%1000000]

  x = new_x
  model = SVC(kernel = 'rbf', C=specs[k][0],gamma = specs[k][1])
  if(k<=-1):
    samples = 50000
    model.fit(x[0:samples],y[0:samples])
  else:
    samples= 100000
    for i in range(10):
      print("iteration ", i)
      model.fit(x[i*samples:(i+1)*samples],y[i*samples:(i+1)*samples])
  
  y_pred = model.predict(x[400000:500000])
  y_true = y[400000:500000]
  print(f1_score(y_true, y_pred, average=None))
  print(f1_score(y_true, y_pred, average='macro'))
  print("appending ", k)
  models.append(model)
  print(k,"   done")

starting  0
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
[0.99974483 0.98767258]
0.993708704439243
appending  0
0    done
starting  1
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
[0.99508513 0.99832366]
0.9967043904302098
appending  1
1    done
starting  2
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
[0.98168154 0.98328774 0.98655711 0.99119365]
0.9856800077570663
appending  2
2    done
starting  3
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
[0.57142857 0.77130045 0.83741648 0.85148148 0.87377758 0.88194973
 0.88713455 0.89299653 0.89414198 0.88910533]
0.8350732676539406
appending  3
3    done
starting  4
iteration  0
iteration  1
iteration  2
iterati

In [11]:
# models[3]=model

IndexError: list assignment index out of range

In [ ]:
model_ref = [0,0,1,2,3,4,1,2,4,3]
y_valid = np.zeros((1000000))
y_pred = np.zeros((1000000))
for k in range(6):
  if(model_ref[k]==2 or model_ref[k]==4):
    lengt = 3
  elif(model_ref[k]==3):
    lengt = 13
  else:
    lengt = 7
  print("starting ", k)
  x = train_signal[k].flatten()
  y = train_channels[k].flatten()

  y = np.array(y).astype(int)
  new_x = np.zeros((x.shape[0],lengt))
  for i in range(x.shape[0]):
        for j in range(-lengt//2,lengt//2 + 1):
            new_x[i][j + lengt//2] = x[(i +j + 500000)%500000]

  x = new_x
  model = models[model_ref[k]]
  y_pred[k*100000:(k+1)*100000] = model.predict(x[400000:500000])
  y_valid[k*100000:(k+1)*100000]=y[400000:500000]

print(f1_score(y_valid, y_pred, average=None))
print(f1_score(y_valid, y_pred, average='macro'))

In [ ]:
model_ref = [1,3,5,1,2,4,5,4,1,3,1,1,1,1,1,1,1,1,1,1]
model_ref = np.array(model_ref).astype(int) - 1
y_pred_all = np.zeros((2000000))
for pec in range(20):
  if(model_ref[pec]==2 or model_ref[pec]==4):
    lengt = 3
  else:
    lengt = 7
  print(pec)
  x_test = test_signal.flatten()[pec*100000:(pec+1)*100000]
  x = x_test
  new_x = np.zeros((x.shape[0],lengt))
  for i in range(x.shape[0]):
        for j in range(-lengt//2,lengt//2 + 1):
            new_x[i][j + lengt//2] = x[(i +j + 100000)%100000]

  x = new_x
  x_test = x
  test_pred = models[model_ref[pec]].predict(x_test)
  y_pred_1 = np.array(test_pred).astype(int)
  y_pred_all[pec*100000:(pec+1)*100000] = y_pred_1

y_pred_all = np.array(y_pred_all).astype(int)

In [ ]:
import pandas as pd
sub = pd.read_csv(data_path + 'sample_submission_ion.csv')

sub.iloc[:,1] = y_pred_all
sub.to_csv(data_path + 'my_submission_svm_5_best_400k_lengt13.csv',index=False,float_format='%.4f')
print("saved the file")